# AoAI Assigment-1
---------------------------

I have provided the codes for Part-1 and Part-2 together in this jupyter file as below. Codes and functions are also explained in the below markdown cells.
Part-1
-------
My custom CSV parser has one main function and two inner functions as below. <br>
> <ins>Main Function</ins>: <b>read_csv</b> :
<br> Main function is the one which we call when a file is intended to be read and parsed. It returns two arrays which include our data and header information respectively. This function can have 5 arguments as shown below: <br><br>
<b>read_csv</b>(filename, encoding="UTF-8-sig", header=0, read_only=False, time_column=None) <br>
- <i>Function Argument Explanations:</i>
    - filename: It takes the CSV file name with its full path
    - encoding: Filename encoding can be passed as an argument, default is "UTF-8-sig".
    - header: If the CSV file has a header, it can be identified by the parser by passing the row number of header, default value is 0, and if no header in the file then None can be passed.
    - read_only: It is passed with True for parser to only read but not parse(print) the file, default value is False
    - time_column: It is used to indicate the time column when we want our parser to convert time data to datetime object format <br> ("0000-00-00 00:00:00"), and if the time data is not in a suitable form then it returns "invalid time", default value is None.<br>
    
> Inner functions are as follows.
>> <ins>Inner Function-1</ins>: <b>date_parser</b> :
<br> This function is optional and can be opt when calling the main function with time_column argument as described in main function section above. It basically checks if the date data is in a valid time format and if so it returns the data with its suitable datetime object ("0000-00-00 00:00:00"), if not then it returns with "Invalid time" <br><br>
>> <ins>Inner Function-2</ins>: <b>split_lines</b> :
<br> This function splits the lines in the file using the delimiter "," and creates a 2-d array with the data provided and returns it.

In [1]:
# import libraries
import numpy as np
import datetime as dt
import os
import re

In [2]:
# Main function to call when a file is intended to be read and parsed 
# It returns two arrays which include our data and header information respectively 
def read_csv(filename, encoding="UTF-8-sig", header=0, read_only=False, time_column=0):
    data_array = []
    header_array = []
    
    # function to check if the date data is in a valid time format and if so it returns
    # the data with its suitable datetime object ("0000-00-00 00:00:00"), if not then 
    # it returns with "Invalid time"
    def date_parser(data_array_time_column):
        # define data formats
        date_formats = ('%Y', '%b %d, %Y', '%b %d, %Y', '%B %d, %Y', '%B %d %Y', '%B.%d.%Y', '%B-%d-%Y', \
                        '%m/%d/%Y', '%m.%d.%Y', '%m-%d-%Y', '%m/%d/%y', '%m.%d.%y', '%m-%d-%y', '%d/%m/%y', \
                        '%d.%m.%y', '%d-%m-%y', '%b %Y','%B %Y','%b %d,%Y')                       
        parsed = ""
        
        # compile the datetime object format
        r = re.compile('.{4}-.{2}-.{2} .{2}:.{2}:.{2}')
        
        # datetime implementation
        for i, date_data in enumerate(data_array_time_column):
            counter = 0            
            if not(r.match(date_data) and len(date_data) == 19):
                for date_format in date_formats:
                    try:
                        time = dt.datetime.strptime(date_data, date_format)
                        data_array_time_column[i] = time
                        break
                    except ValueError as err:
                        counter += 1
                        pass
        
                if counter == 19:
                    data_array_time_column[i] = "Invalid Time"
    
    # function to split the lines in the file using the delimiter ","
    # and create a 2-d array with the data provided and returns it
    def split_lines(data_lines):
        data_list = []
        compile_ = re.compile(r",(?=(?:[""^\"']*[""\"'][""^\"']*[""\"'])*[""^\"']*$)")
        for line in data_lines:            
                                   
            line = [value.strip(' "') for value in compile_.split(line)]
            line_lenghts.append(len(line))
            data_list.append(line)

            max_line_lenghts = max(line_lenghts)

        # add "None" when there are no date provided
        for line in data_list:
            while len(line) < max_line_lenghts:
                line.append("None")
        return data_list
    
    # error handling
    try:
        # parse the file and print the results in a tabular format
        with open(filename, "r", encoding=encoding) as file:
            line_lenghts = []
            max_box_length_list = []
            lines = file.readlines()
            lines = [line.strip("\n") for line in lines]

            full_list = split_lines(lines)
            
            # check if there is a header
            if header == None:
                header = -1
            else:            
                header_list = full_list[header]
                header_array = np.array([header_list])

            data_list = full_list[header+1:]          
            data_array = np.array(data_list)
            
            # check if there is a time_column
            if time_column != None:
                date_parser(data_array[:, time_column])
            
            # if there is a header defined add it to the array for the output
            if header != -1:
                data_array_for_print = np.concatenate((header_array, data_array))
            else:
                data_array_for_print = data_array
            
            # print the output
            if not(read_only):
                for i in range(data_array_for_print.shape[1]):
                    max_box_length = len(max(data_array_for_print[:, i], key=len))
                    max_box_length_list.append(max_box_length)
                
                for j in range(data_array_for_print.shape[0]):                    
                    for value, max_box_length in zip(data_array_for_print[j ,:], (max_box_length_list)) :
                        left_empty_string = (max_box_length - len(value)) * " "                      
                        print(left_empty_string + value, end="    ")                   
                    print()
                print()      
    except (FileNotFoundError, IOError):
        print("File not found, wrong file or filepath \n")
    except :
        print("Something went wrong when reading the file, please double check if it is a valid csv file with a correct format \n")
      
    return data_array, header_array

Part-1 (Continue)
-------
Below, there is a script which takes the files in folder "weather-data" and call our parser function "read_csv" for each of them. As can be seen from the output, our function reads each file and returns their outputs in a clear tabular format that we can easily monitor and evaluate data. At the start of each output, it is indicated which file is parsed.<br>

>Optionally, if we pass <ins>time_column</ins> argument with the "read_csv" function (such as time_column=0) it also detects whether the time data provided is in a correct format and if so returns with a datetime object, if not returns with "Invalid time". In this particular example below "time_column=0" is passed but since there are no different time data format provided within the files in weather-data it just returns the timedata as is.

>If there are time data with different formats within the hidden files, by assigning the "time_column" with the respective column number of the time data, we can have the converted datetime objects appeared in our output.


In [3]:
# Here we parse all files together in a clear tabular format.
file_names = ["indoor-temperature-1617.csv", "outside-temperature-1617.csv",  \
               "test.png", "barometer-1617-copy.png", "rainfall-1617.csv"]

for file_name in file_names:
    file_path = os.path.join("weather-data", file_name)
    print('\033[1m',"\u0332".join("Parsed file name:"), '\033[0m', file_name)
    read_csv(file_path, header=None, time_column=None)
    

 P̲a̲r̲s̲e̲d̲ ̲f̲i̲l̲e̲ ̲n̲a̲m̲e̲:  indoor-temperature-1617.csv
           DateTime    Humidity    Temperature    Temperature_range (low)    Temperature_range (high)    
2016-10-09 00:00:00          54          21.93                         21                        22.8    
2016-10-10 00:00:00          52          21.77                       20.4                        23.6    
2016-10-11 00:00:00          51          21.36                       19.9                          23    
2016-10-12 00:00:00          51          21.44                         20                        23.6    
2016-10-13 00:00:00          52          21.22                       20.1                        22.3    
2016-10-14 00:00:00          52          21.02                       19.6                        22.6    
2016-10-15 00:00:00          53           21.4                       20.3                        22.5    
2016-10-16 00:00:00          53          21.43                         20               

2017-02-14 00:00:00          39          19.81                       18.1                        23.2    
2017-02-15 00:00:00          41          22.07                         20                        24.3    
2017-02-16 00:00:00          42          20.95                       19.4                        23.4    
2017-02-17 00:00:00          43           21.1                       19.8                        23.4    
2017-02-18 00:00:00          43          20.73                       19.5                        22.5    
2017-02-19 00:00:00          43           20.9                       19.5                        23.5    
2017-02-20 00:00:00          45          21.21                       19.9                        23.1    
2017-02-21 00:00:00          46          21.18                       19.6                          23    
2017-02-22 00:00:00          47          20.77                       19.4                        22.9    
2017-02-23 00:00:00          47          20.83

2017-07-06 00:00:00          52          27.06                       25.8                        28.4    
2017-07-07 00:00:00          52          27.19                       26.2                        28.2    
2017-07-08 00:00:00          51          27.06                       26.4                        27.7    
2017-07-09 00:00:00          51          27.61                       26.4                        29.3    
2017-07-10 00:00:00          51          27.07                       26.4                        28.3    
2017-07-11 00:00:00          52          25.17                       24.1                        26.6    
2017-07-12 00:00:00          52          23.76                         23                        24.4    
2017-07-13 00:00:00          52          24.06                       23.5                        24.8    
2017-07-14 00:00:00          52          24.58                         24                        25.6    
2017-07-15 00:00:00          52          24.14

2016-12-02 00:00:00           2.46                       -0.2                         4.4    
2016-12-03 00:00:00           4.22                        1.3                         6.3    
2016-12-04 00:00:00           2.12                        0.3                         4.9    
2016-12-05 00:00:00           2.88                          1                         5.1    
2016-12-06 00:00:00           7.45                        4.9                         9.6    
2016-12-07 00:00:00          10.74                        8.8                          13    
2016-12-08 00:00:00          12.44                       11.8                        13.2    
2016-12-09 00:00:00          12.01                         11                        13.1    
2016-12-10 00:00:00           9.84                        5.6                        11.5    
2016-12-11 00:00:00           5.92                        3.3                         9.2    
2016-12-12 00:00:00              7                          

2017-06-20 00:00:00          25.63                       18.7                        35.9    
2017-06-21 00:00:00          26.06                         18                        38.5    
2017-06-22 00:00:00          19.15                       15.2                        23.9    
2017-06-23 00:00:00          17.45                       14.8                        20.8    
2017-06-24 00:00:00          17.76                       15.3                        22.6    
2017-06-25 00:00:00          16.84                         15                        19.7    
2017-06-26 00:00:00           18.3                       11.9                        24.3    
2017-06-27 00:00:00          18.63                       15.5                          23    
2017-06-28 00:00:00          14.99                       12.9                        16.6    
2017-06-29 00:00:00          13.64                       11.4                        16.4    
2017-06-30 00:00:00          15.37                         1

2017-03-20 00:00:00     6.3    
2017-03-21 00:00:00       0    
2017-03-22 00:00:00     4.3    
2017-03-23 00:00:00       1    
2017-03-24 00:00:00       0    
2017-03-25 00:00:00       0    
2017-03-27 00:00:00       0    
2017-03-28 00:00:00     3.2    
2017-03-29 00:00:00       0    
2017-03-30 00:00:00       0    
2017-03-31 00:00:00       0    
2017-04-01 00:00:00       1    
2017-04-02 00:00:00       0    
2017-04-03 00:00:00       0    
2017-04-04 00:00:00       0    
2017-04-05 00:00:00       0    
2017-04-06 00:00:00       0    
2017-04-07 00:00:00       0    
2017-04-08 00:00:00       0    
2017-04-09 00:00:00       0    
2017-04-10 00:00:00       0    
2017-04-11 00:00:00       0    
2017-04-12 00:00:00       0    
2017-04-13 00:00:00       0    
2017-04-14 00:00:00       0    
2017-04-15 00:00:00       0    
2017-04-16 00:00:00       0    
2017-04-17 00:00:00       0    
2017-04-18 00:00:00       0    
2017-04-19 00:00:00       0    
2017-04-20 00:00:00     1.1    
2017-04-

## Part-2
In order to calculate the statistics on each CSV file, I wrote a function called "stats" as shown in the next cell. This function reads each file without printing the outputs and uses numpy to calculate mean, maximum, minimum and standard deviation and output them in a clear tabular format. 

It only takes one argument as file_names and calculate the statistics and returns the output for each file seperately indicating file names with their statistics.

In [4]:
def stats(file_names):
    for file_name in file_names:
        
        # create file path
        file_path = os.path.join("weather-data", file_name)
        print('\033[1m',"\u0332".join("Parsed file name:"), file_name)
        
        # read a file and return its data_array and header_array
        data_array, header_array = read_csv(file_path, header=0, read_only=True)
        
        # change "None" to "0" within the data array to correctly calculate statistics
        r = range(len(header_array[0, 1:]))
        data_array[data_array == "None"] = 0
        data_array = np.asfarray(data_array[:,1:],float)
        
        # calculate statistics and print outputs
        for i, label in zip(r, header_array[0, 1:]):      
            mean = np.mean(data_array[:, i]).round(2)
            max_ = np.max(data_array[:, i]).round(2)
            min_ = np.min(data_array[:, i]).round(2)
            std_dev = np.std(data_array[:, i]).round(2)
            print("  ",'\033[1m', label, '\033[0m', "\n     Mean:", mean, "\n     Max:", max_, "\n     Min:", min_, \
                  "\n     Std_Dev:", std_dev, '\n')

In [5]:
# define file names and call function stats in order to calculate statistics
file_names = ["indoor-temperature-1617.csv", "outside-temperature-1617.csv", "barometer-1617.csv", "rainfall-1617.csv"]
stats(file_names)

 P̲a̲r̲s̲e̲d̲ ̲f̲i̲l̲e̲ ̲n̲a̲m̲e̲: indoor-temperature-1617.csv
    Humidity  
     Mean: 48.52 
     Max: 59.0 
     Min: 37.0 
     Std_Dev: 5.18 

    Temperature  
     Mean: 21.83 
     Max: 29.21 
     Min: 18.04 
     Std_Dev: 2.06 

    Temperature_range (low)  
     Mean: 20.56 
     Max: 28.2 
     Min: 14.9 
     Std_Dev: 2.4 

    Temperature_range (high)  
     Mean: 23.53 
     Max: 31.1 
     Min: 19.7 
     Std_Dev: 1.7 

 P̲a̲r̲s̲e̲d̲ ̲f̲i̲l̲e̲ ̲n̲a̲m̲e̲: outside-temperature-1617.csv
    Temperature  
     Mean: 11.14 
     Max: 26.38 
     Min: -1.81 
     Std_Dev: 5.35 

    Temperature_range (low)  
     Mean: 7.87 
     Max: 18.7 
     Min: -4.1 
     Std_Dev: 4.87 

    Temperature_range (high)  
     Mean: 15.52 
     Max: 38.5 
     Min: 1.5 
     Std_Dev: 7.02 

 P̲a̲r̲s̲e̲d̲ ̲f̲i̲l̲e̲ ̲n̲a̲m̲e̲: barometer-1617.csv
    Baro  
     Mean: 1010.0 
     Max: 1035.6 
     Min: 979.6 
     Std_Dev: 9.86 

 P̲a̲r̲s̲e̲d̲ ̲f̲i̲l̲e̲ ̲n̲a̲m̲e̲: rainfall-1617.csv
    mm  
 

Part-2 (Continue)
-------
I modified the file “indoor-temperature-1617.csv” by introducing an outlier on its “Temperature” field and saved it with a new name as "indoor-temperature-1617-modified.csv". I replaced one value which was originally “23.93” with “2100.93” that we normally should not read as a value for indoor temperature from a sensor so basically the new value became an outlier.

Below, I called stats function again and calculated the new summary statistics on the modified CSV file.

In [6]:
# define modified file name and calculate new statistics for this one
modified_file_name = ["indoor-temperature-1617-modified.csv"]
stats(modified_file_name)

 P̲a̲r̲s̲e̲d̲ ̲f̲i̲l̲e̲ ̲n̲a̲m̲e̲: indoor-temperature-1617-modified.csv
    Humidity  
     Mean: 48.52 
     Max: 59.0 
     Min: 37.0 
     Std_Dev: 5.18 

    Temperature  
     Mean: 27.7 
     Max: 2100.93 
     Min: 18.04 
     Std_Dev: 110.37 

    Temperature_range (low)  
     Mean: 20.56 
     Max: 28.2 
     Min: 14.9 
     Std_Dev: 2.4 

    Temperature_range (high)  
     Mean: 23.53 
     Max: 31.1 
     Min: 19.7 
     Std_Dev: 1.7 

